In [15]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics

In [2]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def sample_X(size,labels='False'):
    X, y = mnist.train.next_batch(size)
    y = y.astype(int)
    X = Variable(torch.from_numpy(X)).cuda()
    if(labels=='True'):
        return X,y
    else:
        return X

In [19]:
def get_test_data(size):
    X,y = mnist.test.next_batch(size)
    y = y.astype(int)
    y = np.argmax(y,axis=1)
    X = Variable(torch.from_numpy(X)).cuda()
    return X,y

In [5]:
path = '/home/daiict/CVShare/Jeni/hts/bioassay-datasets/'
p_fingerprints = []
labels = []
with open(path+'AID362red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:112])
        labels.append(row[-1])

In [6]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
labels = labels[1:]
print(no_examples,ip_dim)
p_fingerprints[(p_fingerprints==0)] = -1

(3423, 112)


In [7]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)

In [8]:
no_active_ele = (sum(labels2))
print(no_active_ele)

[48]


In [9]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp,self).__init__()
        self.l1 = nn.Linear(784,1000)
        self.l2 = nn.Linear(1000,500)
        self.l3 = nn.Linear(500,200)
        self.l4 = nn.Linear(200,10)
        
    def forward(self,x):
        x = F.tanh(self.l1(x))
        x = F.tanh(self.l2(x))
        x = F.tanh(self.l3(x))
        x = F.sigmoid(self.l4(x))
        
        return x

In [10]:
#weights = torch.cuda.FloatTensor([0,5])
#criterion = nn.CrossEntropyLoss(weight=weights)
criterion = nn.BCELoss()
mymlp = mlp()
mymlp.cuda()
optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
batch_size = 128

In [11]:
def get_train_batch(batch_size):
    samples = np.random.randint(low=0,high=no_examples,size=(batch_size,1))
    train_batch = p_fingerprints[samples].reshape(batch_size,ip_dim)
    train_batch = train_batch.astype(int)
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    #print(labels2[samples])
    target = Variable(torch.cuda.LongTensor(labels2[samples]),requires_grad=False)
    target = target.view(batch_size,1)
    #print(target.type)
    return train_batch,target

In [12]:
for ep in range(1000):
    #train_batch,target = get_train_batch(batch_size)
    train_batch,target = sample_X(batch_size,labels='True')
    target = Variable(torch.cuda.FloatTensor(target),requires_grad = False)
    model_op = mymlp(train_batch)
    #print(model_op.type)
    loss = criterion(model_op,target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if(ep%10==9):
        print(loss.data[0])


0.316274374723
0.294062793255
0.26359051466
0.235411450267
0.226727515459
0.219197899103
0.202346950769
0.200107216835
0.18225209415
0.179489850998
0.161337763071
0.17041002214
0.165559440851
0.144829824567
0.14181303978
0.153606846929
0.146445482969
0.159258455038
0.140667244792
0.137382298708
0.116371050477
0.129782646894
0.117162048817
0.116358116269
0.111007668078
0.122129119933
0.115056857467
0.121797680855
0.111232742667
0.11973760277
0.0972333997488
0.105856060982
0.103682778776
0.101370289922
0.115116812289
0.10467042774
0.0837110430002
0.0925533771515
0.109422780573
0.0930505022407
0.103480994701
0.0866771712899
0.0957250669599
0.0940516740084
0.10272860527
0.0909320339561
0.0889858752489
0.0922249779105
0.0913292318583
0.0980399847031
0.0987862274051
0.0873361453414
0.0813943445683
0.0809206515551
0.0760590061545
0.0956032127142
0.0857269316912
0.08626742661
0.0781341269612
0.0929175168276
0.0943271964788
0.0901827439666
0.083191499114
0.0945921763778
0.0665311664343
0.093128

## Check on MNIST test data

In [27]:
x_test,y_test = get_test_data(size=300)
y_op = mymlp(x_test).cpu().data.numpy()
print(y_op.shape)
y_op = np.argmax(y_op,axis=1)
cf_mat = metrics.confusion_matrix(y_test,y_op)
print(cf_mat)



(300, 10)
[[28  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0  0  0]
 [ 1  0 23  1  1  0  0  1  2  0]
 [ 0  0  1 25  0  0  0  1  0  0]
 [ 0  0  0  0 23  0  0  0  0  0]
 [ 3  0  1  1  0 24  1  0  2  0]
 [ 1  0  0  0  0  1 27  0  0  0]
 [ 1  0  2  1  0  0  0 33  0  2]
 [ 0  0  2  1  0  2  0  0 25  0]
 [ 1  0  0  0  0  0  0  1  0 34]]


In [ ]:
p_fingerprints_whole,tmp_labels = get_train_batch(batch_size=no_examples)
train_op = mymlp(p_fingerprints_whole)
train_op = train_op.cpu().data.numpy()
train_op[train_op>0.5] = 1
train_op[train_op<0.5] = 0
print(sum(train_op))

# Test the model

In [ ]:
path = '/home/daiict/CVShare/Jeni/hts/bioassay-datasets/'
test_fingerprints = []
test_labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        test_fingerprints.append(row[:112])
        test_labels.append(row[-1])

In [ ]:
test_labels2 = np.zeros((len(test_labels),1))
for i,l in enumerate(test_labels):
    if l=='Active':
        test_labels2[i] = 1
    else:
        test_labels2[i] = 0

In [ ]:
test_fingerprints = np.asarray(test_fingerprints[1:])
test_fingerprints = test_fingerprints.astype(int)
no_test_examples = test_fingerprints.shape[0]
print(no_test_examples)
test_fingerprints = Variable(torch.cuda.FloatTensor(test_fingerprints))

In [ ]:
test_op = mymlp(test_fingerprints)

In [ ]:
test_op = test_op.cpu().data.numpy()

In [ ]:
test_labels = test_op

In [ ]:
test_labels[test_op>0.5] = 1
test_labels[test_op<0.5] = 0

In [ ]:
sum(test_labels2)

In [ ]:
sum(test_labels)